In [1]:
import pandas as pd

In [2]:
import torch

In [3]:
torch.cuda.is_available()

True

In [5]:
!pip install scikit-learn

Looking in indexes: https://mirrors.cernet.edu.cn/pypi/web/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 60.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 30.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 86.3 MB/s eta 0:00:00


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
##词袋模型